## MCP Demo Notebook

This notebook demonstrates how to use the MCP clients in Python.

It covers:
- How to directly connect to an MCP server, list available tools, and invoke a tool.
- How to use AI agents (with LangGraph and LangChain) to interact with MCP tools in a conversational way.

Use this notebook as a reference for integrating and experimenting with MCP in your own projects.

**Author:** [Gregory Tan](gregory.tanyj@paynet.my), Senior AI Engineer @ AI R&D

---
### Section 1: Directly Invoking MCP Client

Directly connect to the MCP server, list available tools, and call a tool directly.

---

In [1]:
# (01) Installing Necessary Libraries
!pip install -r requirements.txt -q

In [ ]:
# (01) Importing Necessary Libraries 
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
import os, asyncio, subprocess

In [3]:
# (02) Start MCP server in the background
MCP_SERVER_FILE = "mcp_server.py"
LOG_FILE = "output.log"

# Clean up old processes and logs
os.system(f"pkill -f {MCP_SERVER_FILE}")
open(LOG_FILE, "w").close()

# Start new process
process = subprocess.Popen(
    ["python", MCP_SERVER_FILE],
    stdout=open(LOG_FILE, "w"),
    stderr=subprocess.STDOUT
)

print(f"MCP Server Started | PID {process.pid}")

MCP Server Started | PID 37537


In [4]:
# (02) Check if MCP server is running
!tail -n 20 $LOG_FILE

INFO:     Started server process [37537]
INFO:     Waiting for application startup.
[09/23/25 14:53:52] INFO     StreamableHTTP       ]8;id=289818;file:///opt/anaconda3/envs/demo/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=102728;file:///opt/anaconda3/envs/demo/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#110\110]8;;\
                             session manager                                    
                             started                                            
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


In [5]:
# # (Optional) Stop MCP server
# process.terminate()
# print(f"MCP Server Stopped | PID {process.pid}")

In [6]:
# (03) Interact with MCP Server
headers = {
    # "Authorization": "Bearer sk-1234"  # Auth Header
}

# Connect to server
async with streamablehttp_client("http://127.0.0.1:8000/mcp", headers=headers) as (reader, writer, _):
    async with ClientSession(reader, writer) as session:
        # Initialize connection
        await session.initialize()
        print("Connected to MCP server!\n")

        # # List Tools
        # tools_list = await session.list_tools()
        # print("The tools list are below:")
        # for tool in tools_list.tools:
        #     print(f"- {tool.name}: {tool.description}")
        
        # # Call Tools (get_temperature)
        # result = await session.call_tool("get_temperature", {"city": "kuala lumpur"})
        # print("Calling get_temperature tool:")
        # print("Result: ", result.content[-1].text)

Connected to MCP server!



---
### Section 2: Using AI Agents (LangGraph) to Chat with MCP

This section demonstrates how to use an AI agent (via LangGraph) to interact with MCP tools in a conversational way.

---

In [7]:
# (04) Import libraries for LangGraph
from langgraph.prebuilt import create_react_agent
from langchain_litellm import ChatLiteLLM
from langgraph.checkpoint.memory import InMemorySaver
from langchain_mcp_adapters.client import MultiServerMCPClient

In [ ]:
# (04) Setup LangGraph with LiteLLM and MCP Client
MODEL = "bedrock-claude-3-5-sonnet"
LITELLM_BASE_URL = "your_litellm_url_here"
LITELLM_API_KEY = "your_litellm_key_here"

In [9]:
# (05) Initialize components for the AI agent
model = ChatLiteLLM(
    model=MODEL, # configured in LiteLLM
    api_base=LITELLM_BASE_URL, # URL to the LiteLLM server
    api_key=LITELLM_API_KEY, # API key for LiteLLM
    custom_llm_provider="openai", # mimic OpenAI API
    temperature=0.0, # temperature for the model
    streaming=True, # enable streaming
    verbose=True, # enable verbose logging
)
checkpointer = InMemorySaver()

In [10]:
# (05) Set MCP Tools to LangGraph
client = MultiServerMCPClient(
    {
        "mcp_server": {
            "url": "http://127.0.0.1:8000/mcp",
            "headers": headers,
            "transport": "streamable_http",
        },
    }
)
tools = await client.get_tools()

In [11]:
# (05) Create Agent with MCP Tools
agent = create_react_agent(
    model=model,
    tools=tools,
    prompt="""
    You are a helpful weather assistant. 
    When the user asks for the weather in a specific city, use the tools to find the information. 
    If the tool returns an error, inform the user politely.
    If the tool is successful, present the weather report clearly.
    """,
    checkpointer=checkpointer,
)

In [13]:
# (06) Interact with the agent
# Example 1: Simple interaction
async for response in agent.astream(
    {"messages": [{"role": "user", "content": "hi! i am from kuala lumpur"}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
):
    response["messages"][-1].pretty_print()

================================ Human Message =================================

hi! i am from kuala lumpur
================================== Ai Message ==================================

Hello! It's nice to meet you. I understand that you're from Kuala Lumpur, the capital city of Malaysia. Since you've mentioned your location, would you like to know about the current weather conditions in Kuala Lumpur? I can provide you with information about the temperature and wind speed if you're interested. Just let me know if you'd like me to check that for you.


In [14]:
# Example 2: Weather inquiry
async for response in agent.astream(
    {"messages": [{"role": "user", "content": "What is the weather in my city?"}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
):
    response["messages"][-1].pretty_print()

================================ Human Message =================================

What is the weather in my city?
================================== Ai Message ==================================

Certainly! I'd be happy to check the weather for you in Kuala Lumpur. I'll use our weather tools to fetch the current temperature and wind speed for your city. Let me do that for you right now.
Tool Calls:
  get_temperature (tooluse_w61z2WUGT0q5ahHt6s9-cA)
 Call ID: tooluse_w61z2WUGT0q5ahHt6s9-cA
  Args:
    city: Kuala Lumpur
  get_windspeed (tooluse_n5quki6qRFSo6yZ1hPM2tg)
 Call ID: tooluse_n5quki6qRFSo6yZ1hPM2tg
  Args:
    city: Kuala Lumpur
================================= Tool Message =================================
Name: get_windspeed

{
  "status": "success",
  "report": "The windspeed in Kuala Lumpur is 12.7km/h"
}
================================== Ai Message ==================================

Great! I've got the current weather information for Kuala Lumpur. Here's the report:

T

---
### Section 3: Using MCP Servers from Online

This section shows how to connect to and use MCP servers that are hosted online, not just on your local machine.

---

**VirusTotal MCP**

**Description:**

The VirusTotal MCP server can be used for IOC (Indicator of Compromise) scanning, such as checking file hashes, URLs, IPs, or domains against VirusTotal's threat intelligence database.  
You must provide your own VirusTotal API keys to use this service.

**Repository Link:**  
- [VirusTotal API Key](https://www.virustotal.com/gui/my-apikey)
- [VirusTotal MCP](https://glama.ai/mcp/servers/@BurtTheCoder/mcp-virustotal)

In [ ]:
# (07) Installation of VirusTotal MCP
!npm install -g @burtthecoder/mcp-virustotal

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
changed 40 packages in 4s
⠹
⠹9 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [ ]:
# (07) Set of VirusTotal API Key
VIRUSTOTAL_API_KEY = "your_virustotal_api_key_here"

In [19]:
# (08) Add VirusTotal MCP tool to the existing client
new_client = MultiServerMCPClient(
    {
        "mcp_server": {
            "url": "http://localhost:8000/mcp",
            "headers": headers,
            "transport": "streamable_http",
        },
        "secuity_tools_mcp": {
            "command": "node",
            "args": [f"{subprocess.check_output(["npm", "root", "-g"]).decode().strip()}/@burtthecoder/mcp-virustotal/build/index.js"],
            "env": { "VIRUSTOTAL_API_KEY": VIRUSTOTAL_API_KEY },
            "transport": "stdio",
        },
    }
) 
new_tools = await new_client.get_tools()

In [29]:
# (08) Check the number of tools before and after adding VirusTotal MCP
print("Before: ", len(tools), "| After: ", len(new_tools))

Before:  2 | After:  9


In [30]:
# (09) Create an agent with the new tools
agent_new = create_react_agent(
    model=model,
    tools=new_tools,
    prompt="Use the toots you have to answer the user's questions.",
    checkpointer=checkpointer
)

In [31]:
# (10) Interact with the agent
# Example 3: Updated interaction with VirusTotal tool
async for response in agent_new.astream(
    {"messages": [{"role": "user", "content": "Investigate if this hash safe? 435e67c0fcb0ac34b17754527f264833553ada8fa222aa37a0841b3faf6324a5"}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
):
    response["messages"][-1].pretty_print()   

================================ Human Message =================================

Investigate if this hash safe? 435e67c0fcb0ac34b17754527f264833553ada8fa222aa37a0841b3faf6324a5
================================== Ai Message ==================================

Certainly! I'll investigate the safety of the hash you've provided: 435e67c0fcb0ac34b17754527f264833553ada8fa222aa37a0841b3faf6324a5. This appears to be a SHA-256 hash. I'll use our file analysis tool to check if this file is safe or potentially malicious.
Tool Calls:
  get_file_report (tooluse_i_uRBILzTea7B6RKSLKtfQ)
 Call ID: tooluse_i_uRBILzTea7B6RKSLKtfQ
  Args:
    hash: 435e67c0fcb0ac34b17754527f264833553ada8fa222aa37a0841b3faf6324a5
================================= Tool Message =================================
Name: get_file_report

📁 File Analysis Results
🔑 Hashes:
• SHA-256: 435e67c0fcb0ac34b17754527f264833553ada8fa222aa37a0841b3faf6324a5
• SHA-1: efd04b092e54f150a2cc79f4fe088a770192182f
• MD5: 3e9f28ce8778710c949f73e1d